# Web Scraping

In [20]:
from bs4 import BeautifulSoup
from sqlalchemy import Column, Integer, String, Boolean, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

import requests
import csv
import os

# ruta para guardar los archivos
path = os.getenv('PENGUIN_PATH')

# link de la página
link = 'https://books.toscrape.com/'

## - Clases

In [8]:
class Descarga:
    """
    Clase encargada de manejar la descarga de contenido desde una URL.
    """

    def descargar(self, url:str) -> str:
        """
        Descarga el contenido de texto desde una URL dada.

        Parámetros:
        -----------
        url : str
            Dirección web (URL) desde la cual se quiere obtener el contenido.

        Retorna:
        --------
        str
            El contenido de la respuesta HTTP en formato de texto si la descarga es exitosa.

        Lanza:
        ------
        RuntimeError
            Si ocurre algún error durante la conexión o la descarga.
        """
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.text
        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"ErrorDescarga: {e}")



class Guardar:
    carpeta_contenedora_de_archivos = 'datos'

    """
    Clase para guardar datos obtenidos de una página web en formato CSV.
    """

    def categorias_como_csv(self, filename: str, datos: dict) -> None:
        """
        Guarda categorías y sus enlaces en un archivo CSV.

        Parámetros:
        -----------
        filename : str
            Nombre del archivo (sin extensión).
        datos : dict
            Diccionario con {categoria: link}.
        """
        ruta = self._asegurar_ruta(filename)

        with open(ruta + '.csv', 'w') as archivo:
            print("Writing: categoria,link")
            archivo.write("categoria,link\n")
            for key, value in datos.items():
                print(f"Writing: {key},{value}")
                archivo.write(f"{key},{value}{'\n' if key != list(datos.keys())[-1] else ''}")
            print(f"Message: Datos cargados al archivo {ruta}.csv con éxito.")
            print(f"Closed: Archivo {ruta}.csv")

    def libros_como_csv(self, filename: str, datos: dict) -> None:
        """
        Guarda libros clasificados por categorías en un archivo CSV.

        Parámetros:
        -----------
        filename : str
            Nombre del archivo (sin extensión).
        datos : dict
            Diccionario con {categoria: [ {link: ...}, ... ]}.
        """
        ruta = self._asegurar_ruta(filename)

        with open(ruta + '.csv', 'w') as archivo:
            print("Writing: categoria,link")
            archivo.write("categoria,link\n")
            for key, value in datos.items():
                for item in value:
                    print(f"Writing: {key},{item['link']}")
                    archivo.write(f"{key},{item['link']}{'\n' if key != list(datos.keys())[-1] else ''}")
            print(f"Message: Datos cargados al archivo {ruta}.csv con éxito.")
            print(f"Closed: Archivo {ruta}.csv")

    def informacion_libros_como_csv(self, filename: str, datos: list) -> None:
        """
        Guarda información detallada de libros en un archivo CSV.

        Parámetros:
        -----------
        filename : str
            Nombre del archivo (sin extensión).
        datos : list[dict]
            Lista de diccionarios con información de cada libro.
        """
        ruta = self._asegurar_ruta(filename)

        with open(ruta + '.csv', 'w', encoding='utf-8', errors='replace') as archivo:
            print("Writing: categoria,titulo,precio,en_stock,rating,imagen,descripcion")
            archivo.write("categoria,titulo,precio,en_stock,rating,imagen,descripcion\n")
            for i, data in enumerate(datos):
                d = f"{data['categoria']},{data['titulo']},{data['precio']},{data['en_stock']},{data['rating']},{data['imagen']},{data['descripcion']}"
                print(f"Writing: {d}")
                archivo.write(f"{d}{'\n' if i < len(datos) - 1 else ''}")
            print(f"Message: Datos cargados al archivo {ruta}.csv con éxito.")
            print(f"Closed: Archivo {ruta}.csv")

    def _archivo_existe(self, filename: str) -> bool:
        """
        Verifica si un archivo CSV existe.

        Parámetros:
        -----------
        filename : str
            Ruta del archivo (sin extensión).
        
        Retorna:
        --------
        bool
            True si existe, False en caso contrario.
        """
        return os.path.exists(filename + ".csv")

    def _carpeta_existe(self, carpeta: str) -> bool:
        """
        Verifica si una carpeta existe.

        Parámetros:
        -----------
        carpeta : str
            Ruta de la carpeta.
        
        Retorna:
        --------
        bool
            True si existe, False en caso contrario.
        """
        return os.path.exists(carpeta)

    def _asegurar_ruta(self, filename: str) -> str:
        """
        Garantiza que la carpeta contenedora exista y devuelve la ruta base del archivo.

        Parámetros:
        -----------
        filename : str
            Nombre del archivo (sin extensión).

        Retorna:
        --------
        str
            Ruta completa sin extensión.
        """
        if not self._carpeta_existe(self.carpeta_contenedora_de_archivos):
            os.makedirs(self.carpeta_contenedora_de_archivos)
            print(f"Carpeta creada: {self.carpeta_contenedora_de_archivos}")

        ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        if not self._archivo_existe(ruta):
            print(f"Creating: {ruta}.csv...")
            print(f"Successfully: {ruta}.csv creado con éxito.")
        print(f"Open: {ruta}.csv...")
        return ruta
    

class Scraping():
    """
    Clase para realizar scraping de datos de la web.
    """

    def __init__(self, link:str):
        """
        Inicializa el objeto Scraping.

        Parámetros:
        -----------
        link : str
            URL de la página web a scrappear.
        """
        self.link = link
        self.html = BeautifulSoup(Descarga().descargar(self.link), 'html.parser')
        self._categorias = {}
        self._libros = {}
        self._informacion_libros = []
        
    def buscar_categorias(self) -> None:
        """
        Busca las categorias de la pagina web
        """
        print(f"Connect: {self.link}")
        print("Getting data...")
        categorias = self.html.find('ul', class_='nav nav-list').find_all('li')
        for categoria in categorias:
            categoria_key = f'{categoria.find('a').text.strip()}'
            categoria_link = f'{self.link}/{categoria.find("a").get("href")}'
            print(f"Extracting: Categoria({categoria_key}), link({categoria_link})")
            self._categorias[categoria_key] = categoria_link
        del self._categorias['Books']

    def buscar_libros(self):
        """
        Busca los libros de la pagina web
        """
        for _, (categoria, link) in enumerate(self._categorias.items()):
            full_link = link

            print(f"Connect: {full_link}")
            html = BeautifulSoup(Descarga().descargar(full_link), 'html.parser')
            
            print(f"Getting data...")
            # Busca los libros en el index de cada categoria
            libros = html.find('ol', class_='row').find_all('li')

            tmp_libros = []
            for libro in libros:
                data = f'{self.link}/catalogue{libro.find("a")["href"].split('/..')[-1]}'
                print(f"Extracting: {data}")
                tmp_libros.append({
                    'link': data,
                    })
            self._libros[categoria] = tmp_libros

            tiene_indice = html.find('div', class_='col-sm-8 col-md-9').find('li', class_='current')
            # tiene_indice = html.find('li', class_='current')
            if tiene_indice:
                print(f"Have index...")
                indice = tiene_indice.text.split()[-1]
                print(f"Getting index from {categoria}...")
                for i in range(1, int(indice) + 1):
                    full_link = f'{full_link}?page={i}.html'
                    print(f"Connect: {full_link}")
                    html = BeautifulSoup(Descarga().descargar(full_link), 'html.parser')
        
                    # Busca los libros en el index de cada categoria
                    libros = html.find('ol', class_='row').find_all('li')
                    print("Getting data...")
                    tmp_libros = []
                    for libro in libros:
                        data = f'{self.link}/catalogue{libro.find("a")["href"].split('/..')[-1]}'
                        print(f"Extracting: {data}")
                        tmp_libros.append({
                            'link': data,
                            })
                    self._libros[categoria] = tmp_libros

    def buscar_libro_individual(self):
        """
        Busca los libros de la pagina web
        """
        pass

    def buscar_libros_del_link(self):
        """
        Busca los libros de la pagina web
        """
        for categoria, links in self._libros.items():
            # print(categoria, link)
            for link in links:
                link = link['link']
                print(f"Connect: {categoria} {link}")
                html = BeautifulSoup(Descarga().descargar(link), 'html.parser')

                print("Extracting information...")
                title = html.find('article', class_='product_page').find('h1').text.strip()
                precio = html.find('p', class_='price_color').text.strip()
                en_stock = html.find('p', class_='instock').find('i', class_='icon-ok').text.strip()
                rating = html.find('p', class_='star-rating').attrs['class'][1]
                imagen_link = self.link + '/' + html.find('div', class_='item').find('img').attrs['src'].split('../')[-1]
                existe_etiqueta_descripcion = html.find('div', id='product_description')
                if existe_etiqueta_descripcion and existe_etiqueta_descripcion.next_sibling:
                    descripcion = html.find('div', id='product_description').find_next_sibling('p').text
                else:
                    descripcion = None

                print(f"Data: {categoria}, {title}, {precio}, {en_stock}, {rating}, {imagen_link}, {descripcion}")

                self._informacion_libros.append({
                    'categoria': categoria,
                    'titulo': title,
                    'precio': precio,
                    'en_stock': en_stock,
                    'rating': rating,
                    'imagen': imagen_link,
                    'descripcion': descripcion,
                })
                print(self._informacion_libros)

    def run(self):
        """Ejecuta el programa"""
        self.buscar_categorias()
        Guardar().categorias_como_csv('categorias', self._categorias)
        self.buscar_libros()
        Guardar().libros_como_csv('libros', self._libros)
        self.buscar_libros_del_link()
        Guardar().informacion_libros_como_csv('informacion_libros', self._informacion_libros)



## - Base de Datos

In [9]:
from sqlalchemy import Column, Integer, String, Boolean, ForeignKey
from sqlalchemy.orm import relationship, declarative_base

Base = declarative_base()

class Categoria(Base):
    __tablename__ = 'categorias'
    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    link = Column(String)

    # Relación inversa opcional
    libros = relationship('Libro', back_populates='categoria')

class Libro(Base):
    __tablename__ = 'libros'
    id = Column(Integer, primary_key=True)
    titulo = Column(String)
    precio = Column(String)
    descripcion = Column(String)
    imagen = Column(String)

    id_categoria = Column(Integer, ForeignKey('categorias.id'))
    categoria = relationship('Categoria', back_populates='libros')

    stock = relationship('Stock', back_populates='libro', uselist=False)

class Stock(Base):
    __tablename__ = 'stock'
    id = Column(Integer, primary_key=True)
    en_stock = Column(Boolean)
    cantidad = Column(String)

    id_libro = Column(Integer, ForeignKey('libros.id'))
    libro = relationship('Libro', back_populates='stock')


engine = create_engine('sqlite:///libros.db')

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()


## - Implementación

In [11]:
# scraping = Scraping(link)
# scraping.run()   

## - Insertar datos en la tabla Categoria

In [ ]:
# # insertar datos en la base de datos en la base de datos 'Categorias'

# categorias_ = []
# with open('datos/categorias.csv', 'r') as archivo:
#     reader = csv.reader(archivo)
#     for i, row in enumerate(reader):
#         if i == 0: continue
#         categorias_.append(Categoria(nombre=row[0], link=row[1]))

# session.add_all(categorias_)
# session.commit()
# print("¡Categorias insertadas!")


## - Insertar datos en la tabla libros

In [ ]:
# insertar datos en la base de datos en la base de datos 'Categorias'

categorias_ = []
with open('datos/libros.csv', 'r') as archivo:
    reader = csv.reader(archivo)
    for i, row in enumerate(reader):
        if i == 0: continue
        categorias_.append()

session.add_all(categorias_)
session.commit()
